In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.optimize import curve_fit
from sklearn.linear_model import LogisticRegression
from sklearn import *
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
import statsmodels.api as sm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from scipy.stats import sem
import scipy

In [ ]:
df = pd.read_csv('publication_data.csv')

In [ ]:
data = {}
for i in range(1,9):
    data["Neuron{0}".format(i)] = {}

In [ ]:
for i in range (1,9):
    temp = df[df.Neuron_Index == i]
    dendrite_values = set(temp.Dendrite_Index)
    for j in dendrite_values:
        data["Neuron{0}".format(i)]["Dendrite{0}".format(j)] = {}
        for k in range (1,7):
            temp2 = temp[temp.Dendrite_Index == j]
            data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)] = temp2[temp2.Imaging_Session == k]

In [ ]:
for l in range (1,9):
    temp = df[df.Neuron_Index == l]
    dendrite_values = set(temp.Dendrite_Index)
    data["Neuron{0}".format(l)]["Dendrite_Values"] = np.array(list(dendrite_values))
    for k in dendrite_values:
        spines = np.array([],dtype = 'int')
        for i in range (1,7):
            x = np.array(data["Neuron{0}".format(l)]["Dendrite{0}".format(k)]["Session{0}".format(i)]["Spine_Index"].values)
            x = x.flatten()
            spines = np.append(spines,x)
        spines = set(sorted(spines))
        d = {}
        a = {}
        for i in spines:
            d[i] = [0,0]
        for i in range (1,7):
            x = np.array(data["Neuron{0}".format(l)]["Dendrite{0}".format(k)]["Session{0}".format(i)]["Spine_Index"].values)
            for j in x:
                if d[j][0] == 0:
                    d[j][0] = i
                    d[j][1] = i
                else:
                    d[j][1] = i
                a[j] = d[j][1] - d[j][0]
        data["Neuron{0}".format(l)]["Dendrite{0}".format(k)]["Survival"] = d
        data["Neuron{0}".format(l)]["Dendrite{0}".format(k)]["Age"] = a

In [ ]:
v_all = np.empty([])
for i in range (1,9):
    temp = df[df.Neuron_Index == i]
    dendrite_values = set(temp.Dendrite_Index)
    for j in dendrite_values:
        for k in range (1,7):
                v_all = np.append(v_all,np.array(list(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["V"])).reshape(-1,1))
v_all = v_all[1:].reshape(-1,1)
v_all = StandardScaler().fit_transform(v_all)

In [ ]:
v_all = np.array(sorted(v_all)).reshape(-1,1)
sum(v_all > 1)
v_all = v_all[:-1057]

In [ ]:
v_all

In [ ]:
_, bins, _ = plt.hist(v_all, 100,density = 1)
mu, sigma = scipy.stats.norm.fit(v_all)
best_fit_line = scipy.stats.norm.pdf(bins, mu, sigma)
plt.plot(bins, best_fit_line)

In [ ]:
len(v_all)

In [ ]:
spines_all = np.empty([0,2])
for i in range (1,9):
    dendrite_values = data["Neuron{0}".format(i)]["Dendrite_Values"]
    for j in dendrite_values:
        survivals = data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Survival"]
        spines_all = np.append(spines_all,np.array(list(survivals.values())),axis = 0)

In [ ]:
x = spines_all[spines_all[:,0].argsort()]

In [ ]:
x = sorted(x , key = lambda x : (x[0],-1*x[1]))

In [ ]:
for i in range(len(x)):
    plt.hlines(i+1,(x[i][0]-1) * 4 - 2, (x[i][1]) * 4 - 2)
plt.ylabel("Spine Number")
plt.xlabel("Time (days)")
plt.xticks([0,4,8,12,16,20])

In [ ]:
neuron_count = []
for i in range (1,9):
    temp = df[df.Neuron_Index == i]
    dendrite_values = set(temp.Dendrite_Index)
    count = np.zeros((6,1))
    for j in dendrite_values:
        for k in range (1,7):
                count[k-1] += len(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)])
    neuron_count.append(count)      

In [ ]:
for i in range(8):
    plt.plot(np.linspace(0,5,6),neuron_count[i],marker = 'o')
plt.xticks([0,1,2,3,4,5],[0,4,8,12,16,20])
plt.xlabel("Time (Days)")
plt.ylabel("Number of Spines Imaged")

In [ ]:
x1 = np.array(x[:1420])

In [ ]:
survival = x1[:,1] - x1[:,0]
counts = Counter(survival)

In [ ]:
vals = np.array(list(counts.values()))
survival = []
for i in range(1,6):
    survival = np.append(survival,sum(vals[:-i]))

In [ ]:
survival = np.insert(survival,0,1420)
survival

In [ ]:
prob = survival/1420

In [ ]:
plt.plot(np.linspace(0,5,6)*4,prob,marker='o')
plt.ylim(0,1)
plt.ylabel("Probability of Survival")
plt.xlabel("Time (days)")
plt.xticks([0,4,8,12,16,20])

In [ ]:
x_new = np.array(x[1420:3281])

In [ ]:
imaged = []
cumu = 0
for i in range(2,6):
    a = x_new[:,0] == i
    cumu = cumu + sum(a)
    imaged.append(cumu)

In [ ]:
imaged = sorted(imaged,reverse = True)
imaged

In [ ]:
sessions_survived = x_new[:,1] - x_new[:,0]
sessions_survived

In [ ]:
counts = Counter(sessions_survived)
counts

In [ ]:
vals = np.array(list(counts.values()))
survival = []
for i in range(1,5):
    survival = np.append(survival,sum(vals[:-i]))

In [ ]:
survival
prob = survival/imaged
prob = np.insert(prob,0,1)
prob

In [ ]:
def fun(x,gamma):
    return ((1+x)**(-gamma))

In [ ]:
coeff,_ = curve_fit(fun,np.linspace(0,4,5),prob)

In [ ]:
coeff[0]

In [ ]:
plt.plot(np.linspace(0,4,5),prob,marker = 'o')
plt.plot(np.linspace(0,4,100),fun(np.linspace(0,4,100),1.384),linestyle='dashed')
plt.ylim(0,1)
plt.ylabel("Probability of Survival")
plt.xlabel("Time (days)")
plt.xticks([0,1,2,3,4],[0,4,8,12,16])

In [ ]:
for i in range (1,9):
    temp = df[df.Neuron_Index == i]
    dendrite_values = set(temp.Dendrite_Index)
    for j in dendrite_values:
        for k in range (1,7):
            fo = []
            lo = []
            age = []
            for l in data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["Spine_Index"]:
                fo.append(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Survival"][l][0])
                lo.append(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Survival"][l][1])
                if fo[-1] == 1:
                    age.append(float('inf'))
                else:
                    age.append(k-fo[-1])
            data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["First_Observed"] = fo 
            data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["Last_Observed"] = lo
            data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["Current_Age"] = age
            data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["Sessions_Survived"] = np.array(lo) - np.array(fo)
            l1 = np.array(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["lambda1"])
            l2 = np.array(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["lambda2"])
            data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["S"] = (l1 - l2)/(l1 + l2)

In [ ]:
age_wise = {"age0" : pd.DataFrame(),"age1" : pd.DataFrame(),"age2" : pd.DataFrame(),"age3" : pd.DataFrame(),"older" : pd.DataFrame()}
for i in range (1,9):
    temp = df[df.Neuron_Index == i]
    dendrite_values = set(temp.Dendrite_Index)
    for j in dendrite_values:
        for k in range (1,6):
            for l in range (4):
                x = (data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["Current_Age"] == l)
                age_wise["age{0}".format(l)] = age_wise["age{0}".format(l)].append(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)][x])
            
        x = (data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session5"]["Sessions_Survived"] > 3)
        age_wise["older"] = age_wise["older"].append(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session5"][x])
            

In [ ]:
print(len(age_wise["age0"]),len(age_wise["age1"]),len(age_wise["age2"]),len(age_wise["age3"]),len(age_wise["older"]))

### Age-Size Model

for i in age_wise:
    survival = []
    for index,j in age_wise[i].iterrows():
        if sum(data["Neuron"+str(int(j.Neuron_Index))]["Dendrite"+str(int(j.Dendrite_Index))]["Session"+str(int(j.Imaging_Session + 1))].Spine_Index == j.Spine_Index):
            survival.append(1)
        else:
            survival.append(0)
    age_wise[i]["Survival"] = survival
    vol = np.array(list(age_wise[i]["V"]))
    mean = np.mean(np.array(list(age_wise[i]["V"])))
    std = np.std(np.array(list(age_wise[i]["V"])))
    age_wise[i]["V"] = (vol - mean)/std
    
    shape = np.array(list(age_wise[i]["S"]))
    mean = np.mean(np.array(list(age_wise[i]["S"])))
    std = np.std(np.array(list(age_wise[i]["S"])))
    age_wise[i]["S"] = (shape - mean)/std
    
    dist = np.array(list(age_wise[i]["D"]))
    mean = np.mean(np.array(list(age_wise[i]["D"])))
    std = np.std(np.array(list(age_wise[i]["D"])))
    age_wise[i]["D"] = (dist - mean)/std

bias = []
for i in age_wise:
    x = np.array(age_wise[i]["V"]).reshape(-1,1)
    y = np.array(age_wise[i]["Survival"])
    clf = LogisticRegression(penalty = 'none',random_state=0,max_iter=10000).fit(x, y)
    bias.append(clf.intercept_)
    plt.plot(np.linspace(-2,1).reshape(-1,1),clf.predict_proba(np.linspace(-2,1).reshape(-1,1))[:,-1])
plt.scatter([0,1,2,3,4],bias)    

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

bias_new = []
for i in age_wise:
    x = np.array(age_wise[i]["V"]).reshape(-1,1)
    y = np.array(age_wise[i]["Survival"]).reshape(-1,1)
    number_of_classes = 1
    number_of_features = x.shape[1]
    model = Sequential()
    model.add(Dense(number_of_classes,activation = 'sigmoid',input_dim = number_of_features))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x, y,epochs = 100,verbose = 0)
    bias_new.append(Dense.get_weights(model)[1][0])

bias = []
weights = [[],[],[]]
for i in age_wise:
    x1 = np.array(age_wise[i]["V"]).reshape(-1,1)
    x2 = np.array(age_wise[i]["S"]).reshape(-1,1)
    x3 = np.array(age_wise[i]["D"]).reshape(-1,1)
    x = np.concatenate([x1,x2,x3],axis = 1)
    y = np.array(age_wise[i]["Survival"])
    clf = LogisticRegression(penalty = 'none',random_state=0,max_iter=10000).fit(x, y)
    bias.append(clf.intercept_)
    weights[0].append(clf.coef_[0][0])
    weights[1].append(clf.coef_[0][1])
    weights[2].append(clf.coef_[0][2])
plt.scatter([0,1,2,3,4],bias)    

bias_new = []
weights_new = [[],[],[]]
for i in age_wise:
    x1 = np.array(age_wise[i]["V"]).reshape(-1,1)
    x2 = np.array(age_wise[i]["S"]).reshape(-1,1)
    x3 = np.array(age_wise[i]["D"]).reshape(-1,1)
    x = np.concatenate([x1,x2,x3],axis = 1)
    y = np.array(age_wise[i]["Survival"]).reshape(-1,1)
    number_of_classes = 1
    number_of_features = x.shape[1]
    model = Sequential()
    model.add(Dense(number_of_classes,activation = 'sigmoid',input_dim = number_of_features))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x, y,epochs = 100,verbose = 0)
    bias_new.append(Dense.get_weights(model)[1][0])
    weights_new[0].append(Dense.get_weights(model)[0][0][0])
    weights_new[1].append(Dense.get_weights(model)[0][1][0])
    weights_new[2].append(Dense.get_weights(model)[0][2][0])

In [ ]:
for i in age_wise:
    for j in age_wise:
        if i == j:
            age_wise[i][j] = np.ones(len(age_wise[i]))
        else:
            age_wise[i][j] = np.zeros(len(age_wise[i]))

In [ ]:
age_wise["older"]

In [ ]:
for i in age_wise:
    survival = []
    for index,j in age_wise[i].iterrows():
        if j.Last_Observed > j.Imaging_Session:
            survival.append(1)
        else:
            survival.append(0)
    age_wise[i]["Survival"] = survival

In [ ]:
all_ages = pd.DataFrame()
for i in age_wise:
    all_ages = all_ages.append(age_wise[i])

In [ ]:
all_ages

In [ ]:
x0 = np.array(all_ages["age0"]).reshape(-1,1)
x1 = np.array(all_ages["age1"]).reshape(-1,1)
x2 = np.array(all_ages["age2"]).reshape(-1,1)
x3 = np.array(all_ages["age3"]).reshape(-1,1)
x4 = np.array(all_ages["older"]).reshape(-1,1)
x5 = StandardScaler().fit_transform(np.array(all_ages["V"]).reshape(-1,1))
x = np.concatenate([x0,x1,x2,x3,x4,x5],axis = 1)
y = np.array(all_ages["Survival"])
clf = LogisticRegression(penalty = 'none' , random_state=0 , max_iter=10000 , fit_intercept=False).fit(x, y)

In [ ]:
print(clf.coef_)
log_loss(y, clf.predict_proba(x))

In [ ]:
model = sm.Logit(y, x)
result = model.fit()
result.summary(alpha = .32)


In [ ]:
coef = clf.coef_[0][:-1]
intervals = result.conf_int(0.32)[:-1]
plt.scatter(np.linspace(0,4,5),coef,marker = 'o')
plt.xlabel("Spine Age (days)")
plt.ylabel("Age Coefficients " + "$ b_{i}$")
for i in range(len(intervals)):
    plt.vlines(i,intervals[i][0],intervals[i][1])
    plt.hlines(intervals[i][0],i-0.1,i+0.1)
    plt.hlines(intervals[i][1],i-0.1,i+0.1)
plt.xticks([0,1,2,3,4],[0,4,8,12,16])

In [ ]:
v = np.linspace(-1,3).reshape(-1,1)

ones = np.ones((len(v),1))
zeros = np.zeros((len(v),1))

x1 = np.concatenate([ones,zeros,zeros,zeros,zeros,v],axis = 1)

x2 = np.concatenate([zeros,ones,zeros,zeros,zeros,v],axis = 1)

x3 = np.concatenate([zeros,zeros,ones,zeros,zeros,v],axis = 1)

x4 = np.concatenate([zeros,zeros,zeros,ones,zeros,v],axis = 1)

x5 = np.concatenate([zeros,zeros,zeros,zeros,ones,v],axis = 1)

In [ ]:
v_age0 = list(age_wise["age0"]["V"])
surv_age0 = list(age_wise["age0"]["Survival"])
arr = zip(v_age0,surv_age0)
v_age0,surv_age0 = zip(*sorted(arr, key = lambda x:x[0]))
v_age0 = np.array(v_age0).reshape(-1,1)
surv_age0 = np.array(surv_age0).reshape(-1,1)
vs_age0 = np.concatenate((v_age0,surv_age0),axis = 1)

v_age1 = list(age_wise["age1"]["V"])
surv_age1 = list(age_wise["age1"]["Survival"])
arr = zip(v_age1,surv_age1)
v_age1,surv_age1 = zip(*sorted(arr, key = lambda x:x[0]))
v_age1 = np.array(v_age1).reshape(-1,1)
surv_age1 = np.array(surv_age1).reshape(-1,1)
vs_age1 = np.concatenate((v_age1,surv_age1),axis = 1)

v_age2 = list(age_wise["age2"]["V"])
surv_age2 = list(age_wise["age2"]["Survival"])
arr = zip(v_age2,surv_age2)
v_age2,surv_age2 = zip(*sorted(arr, key = lambda x:x[0]))
v_age2 = np.array(v_age2).reshape(-1,1)
surv_age2 = np.array(surv_age2).reshape(-1,1)
vs_age2 = np.concatenate((v_age2,surv_age2),axis = 1)

v_age3 = list(age_wise["age3"]["V"])
surv_age3 = list(age_wise["age3"]["Survival"])
arr = zip(v_age3,surv_age3)
v_age3,surv_age3 = zip(*sorted(arr, key = lambda x:x[0]))
v_age3 = np.array(v_age3).reshape(-1,1)
surv_age3 = np.array(surv_age3).reshape(-1,1)
vs_age3 = np.concatenate((v_age3,surv_age3),axis = 1)

v_older = list(age_wise["older"]["V"])
surv_older= list(age_wise["older"]["Survival"])
arr = zip(v_older,surv_older)
v_older,surv_older = zip(*sorted(arr, key = lambda x:x[0]))
v_older = np.array(v_older).reshape(-1,1)
surv_older = np.array(surv_older).reshape(-1,1)
vs_older = np.concatenate((v_older,surv_older),axis = 1)

v_agewise = {"age0":vs_age0,"age1":vs_age1,"age2":vs_age2,"age3":vs_age3,"older":vs_older}

In [ ]:
v_avg = np.empty((4,5),dtype = float)
v_std = np.empty((4,5))
prob = np.empty((4,5))
prob_sem = np.empty((4,5))
k = 0
all_vol = np.array(all_ages["V"]).reshape(-1,1)
v_mean = np.mean(all_vol)
v_stdev = np.std(all_vol)
for i in v_agewise:
    vol = v_agewise[i][:,0]
    vol = (vol-v_mean)/v_stdev
    p = v_agewise[i][:,1]
    for j in range (4):
        n = len(vol)
        v_avg[j,k] = np.mean(vol[int(j/4 * n) : int((j+1)/4 * n)-1])
        v_std[j,k] = np.std(vol[int(j/4 * n) : int((j+1)/4 * n)-1])
        prob[j,k] = np.mean(p[int(j/4 * n) : int((j+1)/4 * n)-1])
        prob_sem[j,k] = sem(p[int(j/4 * n) : int((j+1)/4 * n)-1])
    k += 1

In [ ]:
plt.plot(v,clf.predict_proba(x1)[:,1],label = '0-4 days',color = "blue")
plt.plot(v,clf.predict_proba(x2)[:,1],label = '4-8 days',color = "orange")
plt.plot(v,clf.predict_proba(x3)[:,1],label = '8-12 days',color = "green")
plt.plot(v,clf.predict_proba(x4)[:,1],label = '12-16 days',color = "red")
plt.plot(v,clf.predict_proba(x5)[:,1],label = 'older',color = "purple")

ones = np.ones((4,1))
zeros = np.zeros((4,1))
colors = ["blue","orange","green","red","purple"]
for i in range(5):
    plt.scatter(v_avg[:,i],prob[:,i],color = colors[i])
    plt.hlines(prob[:,i],v_avg[:,i] - v_std[:,i],v_avg[:,i] + v_std[:,i],color = colors[i])
    plt.vlines(v_avg[:,i],prob[:,i] - prob_sem[:,i],prob[:,i] + prob_sem[:,i],color = colors[i])

plt.legend()
#plt.xticks([-1.5,-.5,.5,1.5], [0,1,1.5,2])
plt.xlabel("V (a.u)")
plt.ylabel("Probability of Survival")

number_of_classes = 1
number_of_features = x.shape[1]
model = Sequential()
model.add(Dense(number_of_classes,activation = 'sigmoid',input_dim = number_of_features,use_bias = False))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x, y,epochs = 100,verbose = 0)
Dense.get_weights(model)

### Comprehensive  Model

In [ ]:
x0 = np.array(all_ages["age0"]).reshape(-1,1)
x1 = np.array(all_ages["age1"]).reshape(-1,1)
x2 = np.array(all_ages["age2"]).reshape(-1,1)
x3 = np.array(all_ages["age3"]).reshape(-1,1)
x4 = np.array(all_ages["older"]).reshape(-1,1)
x5 = StandardScaler().fit_transform(np.array(all_ages["V"]).reshape(-1,1))
x6 = StandardScaler().fit_transform(np.array(all_ages["S"]).reshape(-1,1))
x7 = StandardScaler().fit_transform(np.array(all_ages["D"]).reshape(-1,1))
x = np.concatenate([x0,x1,x2,x3,x4,x5,x6,x7],axis = 1)
y = np.array(all_ages["Survival"])
clf = LogisticRegression(penalty = 'none',random_state=0,max_iter=10000,fit_intercept=False).fit(x, y)

In [ ]:
model = sm.Logit(y, x)
result = model.fit()
result.summary(alpha = .32)

In [ ]:
coef = clf.coef_[0][:-3]
intervals = result.conf_int(0.32)[:-3]
plt.scatter(np.linspace(0,4,5),coef,marker = 'o')
plt.xlabel("Spine Age (days)")
plt.ylabel("Age Coefficients " + "$ b_{i}$")
for i in range(len(intervals)):
    plt.vlines(i,intervals[i][0],intervals[i][1])
    plt.hlines(intervals[i][0],i-0.1,i+0.1)
    plt.hlines(intervals[i][1],i-0.1,i+0.1)
plt.xticks([0,1,2,3,4],[0,4,8,12,16])

In [ ]:
x = np.array([1,2,3])
plt.bar(x,clf.coef_[0][-3:])
plt.scatter(x,clf.coef_[0][-3:])
plt.vlines(x,result.conf_int(0.32)[-3:][:,0],result.conf_int(0.32)[-3:][:,1])
plt.hlines(result.conf_int(0.32)[-3:][:,1],x+.1,x-.1)
plt.hlines(result.conf_int(0.32)[-3:][:,0],x+.1,x-.1)
plt.xticks([1,2,3], ["$w_{V}$","$w_{S}$","$w_{D}$"])
plt.xlabel("Weights")
plt.ylabel("Morphological Coefficients")

In [ ]:
all_ages.to_excel("Data.xlsx")

v = np.array([])
s = np.array([])
d = np.array([])
for i in range (1,9):
    temp = df[df.Neuron_Index == i]
    dendrite_values = set(temp.Dendrite_Index)
    for j in dendrite_values:
        for k in range (1,7):
            v = np.append(v,np.array(list(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["V"])))
            s = np.append(s,np.array(list(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["S"])))
            d = np.append(d,np.array(list(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["D"])))

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)

mean = np.mean(v)
std = np.std(v)
v = (v - mean)/std

mean = np.mean(s)
std = np.std(s)
s = (s - mean)/std

mean = np.mean(d)
std = np.std(d)
d = (d - mean)/std

##### Normalising the Dataset Completely

v = []
s = []
d = []
for i in range (1,9):
    temp = df[df.Neuron_Index == i]
    dendrite_values = set(temp.Dendrite_Index)
    for j in dendrite_values:
        for k in range (1,7):
            v = np.append(v,np.array(list(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["V"])))
            s = np.append(v,np.array(list(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["S"])))
            d = np.append(v,np.array(list(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["D"])))

v_mean = np.mean(v)
v_std = np.std(v)

s_mean = np.mean(s)
s_std = np.std(s)

d_mean = np.mean(d)
d_std = np.std(d)


for i in range (1,9):
    temp = df[df.Neuron_Index == i]
    dendrite_values = set(temp.Dendrite_Index)
    for j in dendrite_values:
        for k in range (1,7):
            data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["V"] = (data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["V"] - v_mean)/v_std
            data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["S"] = (data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["S"] - s_mean)/s_std
            data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["D"] = (data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)]["D"] - d_mean)/d_std

### Comprehensive Time Dependent Parametric Model

In [ ]:
session2 = pd.DataFrame()
for i in range (1,9):
    temp = df[df.Neuron_Index == i]
    dendrite_values = set(temp.Dendrite_Index)
    for j in dendrite_values:
        session2 = session2.append(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session2"])

In [ ]:
session2

In [ ]:
age0 = []
older = []
for index,j in session2.iterrows():
    if j.Current_Age > 0:
        age0.append(0)
        older.append(1)
    else:
        age0.append(1)
        older.append(0)
session2["age0"] = age0
session2["older"] = older

In [ ]:
survival = []
check_session = 6
for index,j in session2.iterrows():
    if j.Last_Observed >= check_session:
        survival.append(1)
    else:
        survival.append(0)
session2["Survival"] = survival


In [ ]:
x0 = np.array(session2["age0"]).reshape(-1,1)
x1 = np.array(session2["older"]).reshape(-1,1)
x2 = StandardScaler().fit_transform(np.array(session2["V"]).reshape(-1,1))
x3 = StandardScaler().fit_transform(np.array(session2["S"]).reshape(-1,1))
x4 = StandardScaler().fit_transform(np.array(session2["D"]).reshape(-1,1))

x = np.concatenate([x0,x1,x2],axis = 1)
y = np.array(session2["Survival"])
#x = np.ones((len(y),1))
clf = LogisticRegression(penalty = 'none',random_state=0,max_iter=10000,fit_intercept=False).fit(x, y)

In [ ]:
print(clf.intercept_)
print(clf.coef_)
len(x) * log_loss(y, clf.predict_proba(x))

In [ ]:
model = sm.Logit(y, x)
result = model.fit()
result.summary(alpha = .32)

In [ ]:
vol = session2["V"]
size = session2["S"]
dist = session2["D"]
age0 = session2["age0"]
older = session2["older"]
last_observed = session2["Last_Observed"]
arr = zip(age0,older,vol,size,dist,last_observed)
age0,older,vol,size,dist,last_observed = zip(*sorted(arr, key = lambda x:-1*x[-1]))

In [ ]:
for i in range(len(last_observed)):
    plt.hlines(i+1,1,last_observed[i])
plt.ylabel("Spine Number")
plt.xlabel("Days")
plt.xticks([1,2,3,4,5,6], [4,8,12,16,20,24])
plt.title("Empirical Data")

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
age0 = np.array(age0).reshape(-1,1)
older = np.array(older).reshape(-1,1)
vol = StandardScaler().fit_transform(np.array(vol).reshape(-1,1))
size = StandardScaler().fit_transform(np.array(size).reshape(-1,1))
dist = StandardScaler().fit_transform(np.array(dist).reshape(-1,1))

In [ ]:
params = {0:np.array([0.78980183]).reshape(1,-1),
          1:np.array([0.19200686]).reshape(1,-1),
          2:np.array([-0.05413863]).reshape(1,-1),
          3:np.array([-0.30464469]).reshape(1,-1)}
x = np.concatenate([np.ones((len(vol),1))],axis = 1)

In [ ]:
for i in range(len(vol)):
    plt.hlines(i+1,1,2)
for i in range(4):
    z = np.dot(params[i],np.transpose(x))
    y = sigmoid(z)
    for j in range(np.shape(y)[1]):
        plt.hlines(j+1,i+2,i+3,alpha = (y[0][j])/6)
plt.xlabel("Days")
plt.ylabel("Spine Number")
plt.xticks([1,2,3,4,5,6], [4,8,12,16,20,24])
plt.title("Naive Model")

### K-Means Clustering

In [ ]:
x0 = np.array(all_ages["V"]).reshape(-1,1)
x1 = np.array(all_ages["S"]).reshape(-1,1)
x2 = np.array(all_ages["D"]).reshape(-1,1)
x3 = np.array(all_ages["age0"]).reshape(-1,1)
x4 = np.array(all_ages["age1"]).reshape(-1,1)
x5 = np.array(all_ages["age2"]).reshape(-1,1)
x6 = np.array(all_ages["age3"]).reshape(-1,1)
x7 = np.array(all_ages["older"]).reshape(-1,1)

x = np.concatenate([x0,x1,x2,x3,x4,x5,x6,x7],axis = 1)
y = np.array(all_ages["Survival"])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

model = KNeighborsClassifier(n_neighbors = 101)

In [ ]:
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [ ]:
#log_loss(y_test, y_pred, eps=1e-15, normalize=False, sample_weight=None, labels=None)
rep = classification_report(y_test, y_pred,output_dict = True)
print(classification_report(y_test, y_pred))

In [ ]:
rep["macro avg"]["f1-score"]

In [ ]:
errors = []
for i in range (1,333,2):
    model = KNeighborsClassifier(n_neighbors = i)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    errors.append(np.mean(y_pred != y_test))
plt.plot(errors)

In [ ]:
model = KNeighborsClassifier(n_neighbors = 55)
f1 = []
for i in range (1000):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    rep = classification_report(y_test, y_pred,output_dict = True)
    f1.append(rep["macro avg"]["f1-score"])

In [ ]:
np.mean(f1)

### Comprehensive model with complete dataset

In [ ]:
final_data = pd.DataFrame()
for i in range (1,9):
    temp = df[df.Neuron_Index == i]
    dendrite_values = set(temp.Dendrite_Index)
    for j in dendrite_values:
        for k in range(2,4):
            final_data = final_data.append(data["Neuron{0}".format(i)]["Dendrite{0}".format(j)]["Session{0}".format(k)])

In [ ]:
survival = []
for index,j in final_data.iterrows():
    if j.Last_Observed > j.Imaging_Session+2:
        survival.append(1)
    else:
        survival.append(0)
final_data["Survival"] = survival

In [ ]:
age0 = []
older = []
for index,j in final_data.iterrows():
    if j.Current_Age > 0:
        age0.append(0)
        older.append(1)
    else:
        age0.append(1)
        older.append(0)
final_data["age0"] = age0
final_data["older"] = older

In [ ]:
x0 = np.array(final_data["age0"]).reshape(-1,1)
x1 = np.array(final_data["older"]).reshape(-1,1)
x2 = StandardScaler().fit_transform(np.array(final_data["V"]).reshape(-1,1))
x3 = StandardScaler().fit_transform(np.array(final_data["S"]).reshape(-1,1))
x4 = StandardScaler().fit_transform(np.array(final_data["D"]).reshape(-1,1))
x = np.concatenate([np.ones(len(x2)).reshape(-1,1),x2],axis = 1)
y = np.array(final_data["Survival"])
x = np.ones(len(x2)).reshape(-1,1)

In [ ]:
clf = LogisticRegression(penalty = 'none',random_state=0,max_iter=10000,fit_intercept=False).fit(x, y)
print(clf.intercept_)
print(clf.coef_)
from sklearn.metrics import log_loss
log_loss(y, clf.predict_proba(x))

In [ ]:
model = sm.Logit(y, x)
result = model.fit()
result.summary(alpha = .32)

### Neuron wise regression

In [ ]:
all_ages = all_ages.reset_index()
all_ages["Neuron1"] = 0
all_ages["Neuron2"] = 0
all_ages["Neuron3"] = 0
all_ages["Neuron4"] = 0
all_ages["Neuron5"] = 0
all_ages["Neuron6"] = 0
all_ages["Neuron7"] = 0
all_ages["Neuron8"] = 0

In [ ]:
all_ages.columns

In [ ]:
for index,j in all_ages.iterrows():
    all_ages.at[index,"Neuron{0}".format(int(j.Neuron_Index))] = 1

In [ ]:
survival = []
for index,j in all_ages.iterrows():
    if j.Last_Observed > j.Imaging_Session:
        survival.append(1)
    else:
        survival.append(0)
        
all_ages["Survival"] = survival

In [ ]:
x0 = np.array(all_ages["age0"]).reshape(-1,1)
x1 = np.array(all_ages["age1"]).reshape(-1,1)
x2 = np.array(all_ages["age2"]).reshape(-1,1)
x3 = np.array(all_ages["age3"]).reshape(-1,1)
x4 = np.array(all_ages["older"]).reshape(-1,1)
x5 = StandardScaler().fit_transform(np.array(all_ages["V"]).reshape(-1,1))
x6 = StandardScaler().fit_transform(np.array(all_ages["S"]).reshape(-1,1))
x7 = StandardScaler().fit_transform(np.array(all_ages["D"]).reshape(-1,1))
x8 = np.array(all_ages["Neuron1"]).reshape(-1,1)
x9 = np.array(all_ages["Neuron2"]).reshape(-1,1)
x10 = np.array(all_ages["Neuron3"]).reshape(-1,1)
x11 = np.array(all_ages["Neuron4"]).reshape(-1,1)
x12 = np.array(all_ages["Neuron5"]).reshape(-1,1)
x13 = np.array(all_ages["Neuron6"]).reshape(-1,1)
x14 = np.array(all_ages["Neuron7"]).reshape(-1,1)
x15 = np.array(all_ages["Neuron8"]).reshape(-1,1)
x = np.concatenate([x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15],axis = 1)
y = np.array(all_ages["Survival"])

In [ ]:
clf = LogisticRegression(penalty = 'none',random_state=0,max_iter=10000,fit_intercept=False).fit(x, y)
print(clf.intercept_)
print(clf.coef_)
from sklearn.metrics import log_loss
log_loss(y, clf.predict_proba(x))

In [ ]:
model = sm.Logit(y, x)
result = model.fit()
result.summary(alpha = .32)